## Final Project Check-in

## Yolo


## Student Names
1. Dillon Quan
2. Shrikar Thodla
3. Mikio Tada

In [1]:
# import the necessary libraries
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from tqdm import tqdm
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

## Load Data

In [2]:
#reading in the file into pandas dataframe
df = pd.read_csv('data/train.csv')

# separating the target variable and the predictor variables
df_X = df.drop(columns=['hotel_cluster'])
df_y = df['hotel_cluster']

In [3]:
# using stratified shuffle split to get 1% of the original data while keeping the distribution
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.01, random_state=0)
for train_idx, test_idx in sss.split(df_X, df_y):
    X_train, X_test = df_X.iloc[train_idx], df_X.iloc[test_idx]
    y_train, y_test = df_y.iloc[train_idx], df_y.iloc[test_idx]

In [5]:
sample_x = X_train.reset_index(drop=True)
sample_y = y_train.reset_index(drop=True)

In [6]:
sample_x.loc[(sample_x['srch_ci'] > '2017')]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market
58538,2013-02-14 21:54:23,24,2,3,47,25441,NaN,23025,0,1,1,2022-03-13,2023-03-13,3,0,1,15451,3,0,3,6,105,1871


In [7]:
#reset the index
# sample_x = X_train.reset_index(drop=True)
# sample_y = y_train.reset_index(drop=True)

# How many samples are greater than 2017? seems to be only one sample that has the year of 58538 in this 1% sample
#sample_x.loc[(sample_x['srch_ci'] > '2017')] # erroneous year
sample_x.loc[58538, 'srch_ci'] = '2014-11-14'
sample_x.loc[58538, 'srch_co'] = '2014-11-15'

# feature engineering into datetime
sample_x['date_time'] = pd.to_datetime(sample_x['date_time'], yearfirst=True)
sample_x['srch_ci'] = pd.to_datetime(sample_x['srch_ci'], yearfirst=True, errors='coerce')
sample_x['srch_co'] = pd.to_datetime(sample_x['srch_co'], yearfirst=True, errors='coerce')

# timedelta for days in advance feature
sample_x['advance'] = sample_x['srch_ci'] - sample_x['date_time']
sample_x['advance'] = sample_x.advance.dt.days

# duration of stays
sample_x['duration'] = sample_x['srch_co'] - sample_x['srch_ci']
sample_x['duration'] = sample_x.duration.dt.days

In [9]:
new_x = sample_x.drop(columns=['hotel_market', 'user_id', 'site_name',
                      'posa_continent', 'user_location_region', 'user_location_city',
                     'hotel_continent', 'date_time', 'srch_ci', 'srch_co'])

In [10]:
# Train test split the 5% data
xtrain, xtest, ytrain, ytest = train_test_split(new_x, sample_y, test_size=.2 ,random_state=42)

In [11]:
# Building the pipeline
def make_pipeline():
    """This function here makes the pipeline but does not include the clf to prevent data leakage."""
    
    date_features = ['advance', 'duration']
    date_transformer = Pipeline(steps=[('imp', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0))])

    distance = ['orig_destination_distance']
    distance_transformer = Pipeline(steps=[('imp', SimpleImputer(missing_values=np.nan, strategy='mean'))])

    preprocessor = ColumnTransformer(transformers=[('date', date_transformer, date_features),
                                                   ('dist', distance_transformer, distance)],
                                    remainder='passthrough')

    pipeline = Pipeline([('preprocess', preprocessor)])
    
    return pipeline

In [12]:
# make the pipeline and transform the data
pipeline = make_pipeline()
trans_xtrain = pipeline.fit_transform(xtrain)

## Fit scikit-learn model

In [50]:
def make_random_cv(cv, n_iter):
    """
    Steps:
    1. Define Estimator
    2. Define hyperparameter search space
    3. Instantiate RandomizedSearchCV
    """
    
    rf = RandomForestClassifier(random_state=42, oob_score=True)
    params_grid = {'max_features': [0.25, 0.5],
                   'min_samples_leaf': [1000, 15000]}
    
    clf_random_cv = RandomizedSearchCV(estimator=rf, scoring='f1_weighted',
                                       param_distributions=params_grid,
                                       n_iter=n_iter, cv=cv)  
                                                             
                                         
    return clf_random_cv

In [51]:
# creating the RandomizedSearchCV and using the best estimator
randomize_clf = make_random_cv(2, 7) # 2-fold, 7 iterations
randomize_clf.fit(trans_xtrain, ytrain)

/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=7. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/MikioT

/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: Th

## Evaluation Metric

In [52]:
y_train_pred = randomize_clf.predict(trans_xtrain)
r1 = f1_score(ytrain, y_train_pred, average='weighted')
print("Training weighted-f1 score ", r1)
accuracy = accuracy_score(ytrain, y_train_pred)
print("Training accuracy score ", accuracy)

Training weighted-f1 score  0.05090872896935143
Training accuracy score  0.10388205507680158


/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
trans_xtest = pipeline.fit_transform(xtest)
y_test_pred = randomize_clf.predict(trans_xtest)
r1 = f1_score(ytest, y_test_pred, average='weighted')
print("Testing weighted-f1 score ", r1)
accuracy = accuracy_score(ytest, y_test_pred)
print("Testing accuracy score ", accuracy)

Testing weighted-f1 score  0.04996798989478843
Testing accuracy score  0.10213562336576366


/Users/MikioTada/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
